# Using Jamba 1.5 on Bedrock for Standard Operating Procedures and Regulatory Document Analysis
In the pharmaceutical industry, regulatory compliance is crucial for ensuring the safety and efficacy of drug products. Regulatory bodies like the FDA establish detailed guidelines for drug manufacturing processes, which companies must align with through their Standard Operating Procedures (SOPs). Drug compliance officers play a key role in this by reviewing SOPs to identify and rectify any inconsistencies with regulatory standards. Automation tools, such as AI-based comparison systems, can streamline this process by efficiently analyzing and highlighting discrepancies between SOPs and regulatory documents, thereby enhancing compliance accuracy and ensuring the continuous quality of pharmaceutical products.

In this notebook, we use AI21's Jamba 1.5 Large model analyze Subpart E Title 21 of the [Code of Fedral Regulations](https://www.govinfo.gov/app/collection/cfr/2023/title21/chapterI/subchapterC/part211), this section is nearly 30 pages long. We will use Jamba-1.5 Large to perform contradiction detection, to see if a mock Standard Operating Procedure is consistent with the the regulatory document.

In [1]:
# Required imports
import os
import json
import time
import boto3

# Function to read the contents of the files
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()


#Call Bedrock Converse API
def call_bedrock_jamba(user_query, system_prompt):    
    bedrock_client = boto3.client(service_name="bedrock-runtime")
    
    model_id = "ai21.jamba-1-5-large-v1:0" #Jamba 1.5 Large
    #model_id = "ai21.jamba-1-5-mini-v1:0" #Jamba 1.5 Mini
    
    messages = [{
        
        "role": "user", 
        "content": [
                {
                "text": user_query
                },        
            ]
        }
    ]
    
    # Base inference parameters to use.
    inference_config = {"temperature": 0.7, "maxTokens" : 1024}

    # Additional inference parameters to use.
    additional_model_fields = {"top_p": 0.8}

    response = bedrock_client.converse(
        modelId=model_id,
        messages=messages,
        system=system_prompt,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
    
    output_message = response['output']['message']

    print(f"Role: {output_message['role']}")

    for content in output_message['content']:
        print(f"Text: {content['text']}")

    token_usage = response['usage']
    
    print("\n")
    print(f"Input tokens:  {token_usage['inputTokens']}")
    print(f"Output tokens:  {token_usage['outputTokens']}")
    print(f"Total tokens:  {token_usage['totalTokens']}")
 

    return output_message

Read in the SOP and Regulatory Document

In [2]:
# Paths to the text files
sample_sop_path = 'sample_sop.txt'
subpart_path = 'Subpart_E_163_190.txt'

# Read the contents of the files
sample_sop_content = read_file(sample_sop_path)
subpart_content = read_file(subpart_path)


## Prompt Jamba-1.5 model

Next, we will prompt the Jamba-1.5 model to look for specific contradictions. Note that we also are using a system prompt, to have Jamba 1.5 take on the role of a compliance officer.

In [3]:
# System Prompt
system_prompt = [{"text": f"""
    You are a drug compliance officer. You examine Standard Operating Procedures (SOP) to see if they are consistent with regulations.
    """
}]

user_query = f"""
    Explain clearly what the contradiction is, what the SOP says, and how it contradicts the Regulatory Document.

    Standard Operating Procedure: 
    {sample_sop_content}

    Regulatory Document:
    {subpart_content}

    Please analyze the documents documents and determine if the content of Standard Operating Procedure is consistent with the Regulatory Document. 
    Highlight any discrepancies. 

    Specifically, focus on any **blatant contradictions**, where "Standard Operating Procedure" specifies a procedure that directly contradicts the "Regulatory" Document.


    Contradictions Result:
    """

comparison_result = call_bedrock_jamba(user_query, system_prompt)




Role: assistant
Text:  The SOP states that paracetamol can be immediately released to storage and is not required to be quarantined prior to testing. However, the Regulatory Document states that components, drug product containers, and closures shall be stored under quarantine until they have been tested or examined, whichever is appropriate, and released. This is a contradiction.


Input tokens:  30044
Output tokens:  68
Total tokens:  30112


## Conclusion
As you can see from the results above, Jamba 1.5 Large correctly picks up on the fact that in the SOP, Paracetamol is relesed to storage and not quarantined prior to being tested. This flatly contradicts the Regulatory document that requires that the drug product be quarantied prior to testing.  

(note that you may see different results yourself, depending on the temperature you use when calling Jamba)